Introductie

P1

P2

P3

P4

## De invloed van klimaat en geografie op voedselproductie

Niet elk land beschikt over dezelfde natuurlijke omstandigheden om voedsel te produceren. Factoren zoals neerslag, temperatuur en klimaatzones bepalen welke gewassen efficiënt kunnen groeien. In landen met een ongunstig klimaat moeten gewassen onder intensievere en vaak minder duurzame omstandigheden worden verbouwd.
Daarnaast speelt de geografische ligging een rol in hoe voedsel wordt verhandeld: landen die ver van exportmarkten liggen, hebben hogere transportkosten en extra uitstoot, wat hun voedselsysteem minder duurzaam maakt.
In dit perspectief onderzoeken we hoe deze geografische factoren — natuurlijke omstandigheden en ligging ten opzichte van wereldmarkten — bijdragen aan verschillen in de duurzaamheid van nationale voedselsystemen.

#### Klimaatzones en natuurlijke omstandigheden beïnvloeden duurzaamheid
Landen verschillen sterk in klimaat. Tropische regio’s kennen bijvoorbeeld hoge temperaturen en regelmatige regenval, terwijl droge of koude gebieden kampen met beperkte groeiseizoenen en lagere landbouwopbrengsten per hectare.
In warmere, vruchtbare klimaatzones kunnen gewassen natuurlijk groeien, met weinig extra input. In koudere of drogere gebieden zijn irrigatie, verwarming, kunstmest of import van hulpbronnen nodig om dezelfde opbrengst te halen. Deze extra inputs verhogen de uitstoot per kilogram geproduceerd voedsel.
Daarom is het voedselsysteem in sommige landen per definitie minder duurzaam, simpelweg door hun ligging in een ongunstige klimaatzone.